In [ ]:
!pip install optimum
!git clone https://github.com/JustinLin610/AutoGPTQ.git & cd AutoGPTQ
!pip install -v .

In [ ]:
!git clone https://github.com/QwenLM/Qwen-VL.git

In [ ]:
# !cd Qwen-VL
# pwd
!pip install -r /content/Qwen-VL/requirements.txt

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from PIL import Image
import os
excel_path = '/content/drive/MyDrive/meme_gemini_gptv4.xlsx'
image_folder = '/content/drive/MyDrive/meme_abdullah'

# Load the Excel file
df = pd.read_excel(excel_path)

# Ensure the 'llava' column exists
if 'qwenlm' not in df.columns:
    df['qwenlm'] = None

# Iterate over each row in the dataframe
for index, row in df.iterrows():
    meme_id = row['meme_id']
    image_path = os.path.join(image_folder, f"{meme_id}.jpg")

    # Check if the image file exists
    if os.path.exists(image_path):
        # Read the image
        with open(image_path, 'rb') as image_file:
            image_data = image_file.read()

        # Process the image with your model
        image_tag = f'<img>{image_path}</img>'
        response, history = model.chat(tokenizer, query=f'{image_tag} 这是什么', history=None)

        # Save the generated text to the 'llava' column
        print(response)
        df.at[index, 'llava'] = response

        # Save the updated dataframe to the Excel file after every iteration
        with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            df.to_excel(writer, index=False, sheet_name='Sheet1')

print("Process completed.")

In [ ]:
#####################

In [ ]:
!pip install --upgrade optimum
!pip install --upgrade auto-gptq
!pip install --upgrade transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
torch.manual_seed(1234)

# Note: The default behavior now has injection attack prevention off.
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-VL-Chat-Int4", trust_remote_code=True)

# use cuda device
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-VL-Chat-Int4", device_map="cuda", trust_remote_code=True).eval()

# 1st dialogue turn
# query = tokenizer.from_list_format([
#     {'image': 'https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg'},
#     {'text': '这是什么'},
# ])
# response, history = model.chat(tokenizer, query=query, history=None)
# print(response)
# # 图中是一名年轻女子在沙滩上和她的狗玩耍，狗的品种可能是拉布拉多。她们坐在沙滩上，狗的前腿抬起来，似乎在和人类击掌。两人之间充满了信任和爱。

# # 2nd dialogue turn
# response, history = model.chat(tokenizer, '输出"击掌"的检测框', history=history)
# print(response)
# # <ref>击掌</ref><box>(517,508),(589,611)</box>
# image = tokenizer.draw_bbox_on_latest_picture(response, history)
# if image:
#   image.save('1.jpg')
# else:
#   print("no box")


In [ ]:
query = tokenizer.from_list_format([
    {'image': 'https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg'},
    {'text': 'explain the meme in english'},
])
response, history = model.chat(tokenizer, query=query, history=None)
print(response)
# 图中是一名年轻女子在沙滩上和她的狗玩耍，狗的品种可能是拉布拉多。她们坐在沙滩上，狗的前腿抬起来，似乎在和人类击掌。两人之间充满了信任和爱。

# 2nd dialogue turn
# response, history = model.chat(tokenizer, 'explain the meme in english', history=history)
# print(response)
# # <ref>击掌</ref><box>(517,508),(589,611)</box>
# image = tokenizer.draw_bbox_on_latest_picture(response, history)
# if image:
#   image.save('1.jpg')
# else:
#   print("no box")

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from PIL import Image
import os
excel_path = '/content/drive/MyDrive/meme_gemini_gptv4.xlsx'
image_folder = '/content/drive/MyDrive/meme_abdullah'
count = 0
# Load the Excel file
df = pd.read_excel(excel_path)

# Ensure the 'llava' column exists
if 'qwenlm' not in df.columns:
    df['qwenlm'] = None

# Iterate over each row in the dataframe
for index, row in df.iterrows():
    if pd.notna(row['qwenlm']):
      print("skip")
      continue

    meme_id = row['meme_id']
    image_path = os.path.join(image_folder, f"{meme_id}.jpg")

    # Check if the image file exists
    if os.path.exists(image_path):
        # Read the image
        with open(image_path, 'rb') as image_file:
            image_data = image_file.read()

        # Process the image with your model
        image_tag = f'<img>{image_path}</img>'
        response, history = model.chat(tokenizer, query=f'{image_tag} The bot is provided with an image that is a meme. Bot has to provide a summary of the meme, capturing its humour and what makes it humorous. The summary should be clear and engaging', history=None)

        # Save the generated text to the 'llava' column
        print(response)
        print(count)
        count =count + 1
        df.at[index, 'qwenlm'] = response

        # Save the updated dataframe to the Excel file after every iteration
        with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            df.to_excel(writer, index=False, sheet_name='Sheet1')

print("Process completed.")